# Few-Shot Prompts Analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd

from src.pgr_experiment import *
from src.utils import choose_indices
from src.config import MODELS_LIST
from src.math_dataset import load_questions

In [3]:
train_ds = load_questions("train", limit=200)
test_ds = load_questions("test", limit=200)
# # determine k here
# indices = choose_indices(len(train_ds), 3, seed=42)

In [5]:
results = {}
logs = {}
for k in range(0, 6, 2):
    print(f"Running experiments with k={k} few-shots...")
    indices = choose_indices(len(train_ds), k, seed=42)
    pgrs = []
    logs[k] = {}
    for i in range(5):
        pgr, acc_strong_weak, acc_strong_gold, acc_weak_gold = await run_pgr_experiment(
            "gpt-4.1-mini",
            "gpt-5-mini",
            train_ds,
            test_ds[:50],
            indices,
            verbose=True
        )
        pgrs.append(pgr)
        logs[k][i] = {
            "pgr": pgr,
            "acc_strong_weak": acc_strong_weak,
            "acc_strong_gold": acc_strong_gold,
            "acc_weak_gold": acc_weak_gold
        }
    mean_pgr = sum(pgrs) / len(pgrs)
    std_pgr = np.std(pgrs, ddof=1)
    print(pgrs)
    print(f"Mean PGR: {mean_pgr}, Std: {std_pgr}")
    results[k] = {"mean": mean_pgr, "std": std_pgr}

Running experiments with k=0 few-shots...
Acc (strong model with weak labels): 0.9400
Acc (strong model with gold labels): 0.9400
Acc (weak model with gold labels): 0.9200
Acc (strong model with weak labels): 0.9600
Acc (strong model with gold labels): 0.9000
Acc (weak model with gold labels): 0.9200
Acc (strong model with weak labels): 0.9600
Acc (strong model with gold labels): 0.9600
Acc (weak model with gold labels): 0.9200
Acc (strong model with weak labels): 0.9000
Acc (strong model with gold labels): 0.9400
Acc (weak model with gold labels): 0.9200
Acc (strong model with weak labels): 0.9400


CancelledError: 

In [ ]:
with open("results/few_shot_performance.json", "w") as f:
    json.dump(results, f)